In [4]:
# import of libraries
#import bgcArgoDMQC
# NO LONGER USED from bgcArgoDMQC.io import netcdf as CGnetcdf
import datetime
import glob
import gsw
import os
import netCDF4 
import shutil
import numpy as np
import numpy.ma as ma
import pandas as pd
import regex as re
from scipy.interpolate import CubicSpline
import pdb

# what is a BR vs BDfile
# file checker link and script? 
# how to tar and send to AOML DAC 
Make a copy of the BR files and then have a working version of the BD files, then we go into the working BD files and add new variables where we need them. We don’t risk populating the initial data this way. Add new variables, BGC variables, new data. Updating history, meta data, variables, and actual variables. 

The BD files are then run through a file checker in Java. Run the Java file checker, required by the GDAC. 

AOML DAC then sends them to iFEMER (GDAC)

In [2]:
# Global variables & settings
# library settings

np.set_printoptions(threshold=np.inf);

#modify these 3
inst_float = 'aoml_apex'
variables = ['DOXY', 'PH_IN_SITU_TOTAL', 'NITRATE']
#double check this 
profile_var_qc = 'A' 

In [3]:
# select institution and float type
#inst_float = 'pmel_apex' 
#             'pmel_navis'
#             'aoml_apex'
#             'aoml_navis'
inst_float = 'aoml_apex'

floatid = -999999 # flag value, *should* be overwritted by actual WMO ID # later in script 

# change this if the profile is not "overall good":
profile_var_qc = 'A' 
''' ARGO USers manual 3.2.2 Reference table 2a: overall profile quality flag
https://vocab.nerc.ac.uk/collection/RP2/current
N is defined as the percentage of levels with good data where:
● QC flag values of 1, 2, 5, 8 (only used in S-files) are GOOD data
● QC flag values of 0 (no QC), 9 (missing) or “ “ (FillValue) are NOT USED in the computation
● QC flag values of 3, 4 are BAD data
A N = 100%; All profile levels contain good data.
B 75% <= N < 100%
C 50% <= N < 75%
D 25% <= N < 50%
E 0% < N < 25%
F N = 0%; No profile levels have good data
'''

#variables = ['DOXY', 'PH_IN_SITU_TOTAL', 'NITRATE']
variables = [ 'PH_IN_SITU_TOTAL', 'NITRATE']

iprof = {}

# each must be 256 characters or less:
sci_calib_comment = {}
sci_calib_eqn = {}
history_reference = {}

In [4]:
# institution and float specific settings
# PMEL, APEX floats (testing only)
if inst_float == 'pmel_apex':
    raise ValueError('notebook not adapted for pmel_apex!!')
    floatid = 5906293
    # directory settings
    float_dir = f"/raid/frenzel/DMQC/ARGO_PROCESSING/DATA/ARGO_REPO/{floatid}/QC/"
    #format: PRIMARY | https://orcid.org/0000-0001-5766-1668 | Tanya Maurer, MBARI
    comment_dmqc_operator = "PRIMARY | https://orcid.org/0000-0002-3481-0867 | Hartmut Frenzel" 

    # must be 256 characters or less:
    scientific_calibration_comment = 'Polynomial calibration coefficients were used. G determined by comparison of O2sat surface measurement to WOA18; DOXY_ADJUSTED_ERROR is set to 2 mbar. See Takeshita et al., 2013, doi:10.1002/jgrc.20399'

    history_institution = "PM"  # PM for PMEL
    history_reference = "WOA18" # FIXME not really true since they have in-air calibration, but this is for testing only!

    doxy_adj_err = 2 # in mbar, for floats with in-air calibration
    
    # FIXME - is this always true?
    #NEEDS TO BE CHANGED iprof = 1 # use the second entry in the N_PROF dimension

In [5]:
# institution and float specific settings
# PMEL, Navis floats
if inst_float == 'pmel_navis':
    #floatid = 4903499
    floatid = 4903500
    float_dir = f"/raid/frenzel/DMQC/ARGO_PROCESSING/DATA/ARGO_REPO/{floatid}/QC/"
    #format: PRIMARY | https://orcid.org/0000-0001-5766-1668 | Tanya Maurer, MBARI
    comment_dmqc_operator = "PRIMARY | https://orcid.org/0000-0002-3481-0867 | Hartmut Frenzel, PMEL"
    
    for var in ['NITRATE', 'PH_IN_SITU_TOTAL']:
        sci_calib_comment[var] = 'DMQC follows Maurer et al., 2021 (https://doi.org/10.3389/fmars.2021.683207)'
        history_reference[var] = "ESPER-MIX"
    for var in ['DOXY']:
        history_reference[var] = 'WOA'
    
    doxy_adj_err_mbar = 5 # in mbar, for floats without in-air calibration

    history_institution = "PM"  # PM for PMEL
             
    #iprof_doxy = 0 # needed for computation of adjusted error of nitrate and pH
    for var in ['DOXY', 'PH_IN_SITU_TOTAL']:
        iprof[var] = 0 # use the first entry in the N_PROF dimension
    iprof['NITRATE'] = 1 # use the second entry in the N_PROF dimension 

Change float id and comment_dmqc_opperatior and scientific_calibration_comment
    if you have to change 

In [6]:
# institution and float specific settings
# AOML, APEX floats
if inst_float == 'aoml_apex':
    #raise ValueError('notebook not adapted for aoml_apex!')
    floatid = 4903622
    
    float_dir = f"/Users/madison.soden/Documents/MATLAB/ARGO_PROCESSING/DATA/ARGO_REPO/{floatid}/QC/"
    # for Binder float_dir = f"../{floatid}/QC/" 
    # BGCARGO_work/data/{floatid}/"
    #DnetCDF_path = /Users/madison.soden/BGCARGO_work/data/reference/netcdf/5905988/BD5905988_012.nc"
    #RnetCDF_path1 = "/Users/madison.soden/BGCARGO_work/data/4903622/BR4903622_001.nc"
    #RnetCDF_path2 = "/Users/madison.soden/BGCARGO_work/data/4903622/BR4903622_002.nc"
    #format: PRIMARY | https://orcid.org/0000-0001-5766-1668 | Tanya Maurer, MBARI
    comment_dmqc_operator = "PRIMARY | https://orcid.org/0009-0001-9960-129X | Madison Soden, AOML" 
    
    for var in ['NITRATE', 'PH_IN_SITU_TOTAL']:
        sci_calib_comment[var] = 'DMQC follows Maurer et al., 2021 (https://doi.org/10.3389/fmars.2021.683207)'  # must be 256 characters or less
        history_reference[var] = "ESPER-NN"
    for var in ['DOXY']:
        history_reference[var] = 'NCEP' #WOA for no in-air claibration, NCEP for in-air calibration
        sci_calib_comment[var] = 'Polynomial calibration coefficients were used. G determined by surface measurement comparison to NCEP'  # must be 256 characters or less

    
    history_institution = "AO" # AO for AOML
    
    doxy_adj_err_mbar = 2 # in mbar, for floats with in-air calibration
    
    #iprof_doxy = 0 # needed for computation of adjusted error of nitrate and pH
    for var in ['DOXY', 'PH_IN_SITU_TOTAL', 'NITRATE']:
        iprof[var] = 1 # use the second entry in the N_PROF dimension
    

In [7]:
# institution and float specific settings
# AOML, APEX floats
if inst_float == 'aoml_navis':
    floatid = 7901009
    
    float_dir = f"/Users/madison.soden/Documents/MATLAB/ARGO_PROCESSING/DATA/ARGO_REPO/{floatid}/QC/"
    #DnetCDF_path = /Users/madison.soden/BGCARGO_work/data/reference/netcdf/5905988/BD5905988_012.nc"
    #RnetCDF_path1 = "/Users/madison.soden/BGCARGO_work/data/4903622/BR4903622_001.nc"
    #RnetCDF_path2 = "/Users/madison.soden/BGCARGO_work/data/4903622/BR4903622_002.nc"
    #format: PRIMARY | https://orcid.org/0000-0001-5766-1668 | Tanya Maurer, MBARI
    comment_dmqc_operator = "PRIMARY | https://orcid.org/0009-0001-9960-129X | Madison Soden, AOML" 
    
    for var in ['NITRATE', 'PH_IN_SITU_TOTAL']:
        sci_calib_comment[var] = 'DMQC follows Maurer et al., 2021 (https://doi.org/10.3389/fmars.2021.683207)'  # must be 256 characters or less
        history_reference[var] = "ESPER-NN"
    for var in ['DOXY']:
        history_reference[var] = 'WOA' #WOA for no in-air claibration, NCEP for in-air calibration
        sci_calib_comment[var] = 'Polynomial calibration coefficients were used. G determined by surface measurement comparison to NCEP'  # must be 256 characters or less

        
    history_institution = "AO" # AO for AOML
    
    doxy_adj_err = 5 # in mbar, for floats with in-air calibration
    
    for var in ['DOXY', 'PH_IN_SITU_TOTAL']:
        iprof[var] = 0 # use the first entry in the N_PROF dimension
    iprof['NITRATE'] = 1 # use the second entry in the N_PROF dimension 

In [8]:
if floatid < 0:
    raise ValueError('You must select one of the pre-defined institution/float cases')

In [9]:
# other settings that should apply to all institutions and floats
odv_filename = float_dir + f"ODV{floatid}QC.TXT"

data_state_indicator = ['2','C','','']
    # (Ref: Argo Users Manual pgs 85-86) Effectively most cases will change this value from 2B -> 2C for Dmode processing

parameter_data_mode = "D" 
    # either "R" for real time, or "D" for delayed mode

reference_depth = 1500

idx_prof_failure = 100000 # aka "never"

# must be 256 characters or less
sci_calib_eqn = {}
sci_calib_eqn['DOXY'] = 'DOXY_ADJUSTED=DOXY*G;G=G_INIT+G_DRIFT(JULD_PROF-JULD_INIT)/365'

sci_calib_eqn['NITRATE'] = 'NITRATE_ADJUSTED=[NITRATE-[OFFSET + DRIFT(JULD-JULD_PIVOT)/365]]/GAIN' # MBARI version

sci_calib_eqn['PH_IN_SITU_TOTAL'] = 'PH_IN_SITU_TOTAL_ADJUSTED=[PH_IN_SITU_TOTAL+[PUMP_OFFSET - [OFFSET + DRIFT(JULD-JULD_PIVOT)/365]*TCOR]]/GAIN;TCOR=(TREF+273.15)./(TEMP+273.15);  TREF = TEMP at 1500m'



In [10]:
def get_gain_offset_drift_odvqc(filename, variable):
    '''This function retrieves the gain, drift, and offset values
    for the specified variable, each beginnning with a specific profile.
    Returns a dataframe with cycle(station), gain, offset, and drift values.'''
    if variable == 'DOXY':
        odv_var = 'Oxygen'
    elif variable == 'NITRATE':
        odv_var = 'Nitrate'
    elif variable == 'PH_IN_SITU_TOTAL':
        odv_var = 'pH'
    else:
        raise ValueError(f'not coded for variable {variable}')
    with open(filename, 'r') as f_in:
        lines = f_in.readlines()
    # find the segment with QC correction values
    for count, line in enumerate(lines):
        if line.startswith('//QUALITY CONTROLLED DATA CORRECTIONS:'):
            break
    # now find the line(s) for the specified variable
    df = pd.DataFrame(columns = ['Station', 'Gain', 'Offset', 'Drift'])
    for line in lines[count+2:]: # header line comes after "//QUALITY CONTROLLED DATA CORRECTIONS" line
        if line.startswith('//' + odv_var):
            values = line.split() # creates a list of strings
            # skip the first element, which is "//<odv_var>"
            num_values = [float(i) for i in values[1:]]
            df.loc[len(df.index)] = num_values
        elif not line.startswith('//'):
            break # end of QC section reached

    if df.iloc[-1]['Drift'] != 0.0:
        raise ValueError('last drift value should be zero!')
    return df

In [11]:
def create_working_bd_file(filename):
    '''Create a working copy of the B file with a leading 'w_' in the name.
    Note that a possibly existing working copy will be overwritten without
    warning.
    Return the name (with full path) of the working copy.'''
    path, name = os.path.split(filename)
    bd_name = name.replace('BR', 'BD') # no effect if file is named BD*nc already
    w_filename = f'{path}/w_{bd_name}'
    shutil.copyfile(filename, w_filename)
    return w_filename

In [12]:
def get_phys_filename(bgc_filename):
    '''Determine the name of the physical profile file that corresponds
    to the given bgc profile file and return it.
    Throws an IOError if no corresponding file is found.'''
    path, name = os.path.split(bgc_filename)
    # it doesn't matter here if bgc file is named BR* or BD*
    phys_filename = path + '/' + name.replace(name[0:2], 'D') # try D file first
    if not os.path.exists(phys_filename):
        phys_filename = path + '/' + name.replace(name[0:2], 'R') # try R file if necessary
        print(f'Using phys R file:{phys_filename}')
        if not os.path.exists(phys_filename):
            raise IOError(f'No corresponding phys file found for {bgc_filename}')
    return phys_filename        

In [13]:
def get_phys_raw_pres(phys_filename):
    '''Extract the raw PRES values from the given physical profile. 
    Return the full array.
    Pre: File must exist; no check is performed.'''
    phys_file = netCDF4.Dataset(phys_filename, 'r') # read-only access is enough
    pres_raw = phys_file.variables["PRES"][:,:] # for comparison with PRES from BD file
    phys_file.close()
    return pres_raw

In [14]:
def get_dens(phys_filename, verbose=False):
    '''Extract pTS (adjusted if available) from the physical profile file
    with the given name. Compute and return density. Also return salinity and temperature.
    Pre: File must exist; no check is performed.
    FIXME: So far, only existence of adjusted pTS is checked, not the validity
    of the values. (If at least one of them is all NaNs, computed density
    will be all NaNs as well.)'''
    phys_file = netCDF4.Dataset(phys_filename, 'r') # read-only access is enough    
    num_adj = 0
    try:
        temp_var = phys_file.variables["TEMP_ADJUSTED"]
        num_adj += 1
    except KeyError:
        print('TEMP_ADJUSTED not found, using TEMP instead to determine density')
        temp_var = phys_file.variables["TEMP"]
    try:
        psal_var = phys_file.variables["PSAL_ADJUSTED"]
        num_adj += 1
    except KeyError:
        print('PSAL_ADJUSTED not found, using PSAL instead to determine density')
        psal_var = phys_file.variables["PSAL"]
    try:
        pres_var = phys_file.variables["PRES_ADJUSTED"]
        num_adj += 1
    except KeyError:
        print('PRES_ADJUSTED not found, using PRES instead to determine density')
        temp_var = phys_file.variables["PRES"]
    if num_adj == 3:
        if verbose:
            print('Using ADJUSTED values of p,T,S to determine density')
    elif num_adj == 0:
        print('Using RAW values of p,T,S to determine density')
    else:
        print(f'Found {num_adj} variables - case not coded!')
        raise IOError('Unexpected file contents')
    temp = temp_var[:,:]
    psal = psal_var[:,:]
    pres = pres_var[:,:]
    phys_file.close()
    return gsw.rho_t_exact(psal, temp, pres), psal, temp

In [15]:
def organize_b_files(bd_files, br_files):
    '''Sort the B*nc files by profile. If both BR and BD files exist for a given profile,
    consider only the BD file. Return the sorted list.'''
    ptr_bd = 0
    ptr_br = 0    
    
    max_bd = get_profile(bd_files[-1]) if bd_files else 0 #max_bd = 0 if bd_files is empty 
    max_br = get_profile(br_files[-1])
    max_prof = max(max_bd, max_br)
    sorted_b_files = []
    # 001 is the first actual profile file
    for idx in range(1, max_prof+1):
        file_found = False
        for ptr in range(ptr_bd, len(bd_files)):
            if get_profile(bd_files[ptr]) == idx:
                sorted_b_files.append(bd_files[ptr])
                ptr_bd = ptr+1
                file_found = True
                break
        if not file_found:
            for ptr in range(ptr_br, len(br_files)):
                if get_profile(br_files[ptr]) == idx:
                    sorted_b_files.append(br_files[ptr])
                    ptr_br = ptr+1
                    break
        # it is possible that neither BD nor BR file exist for an index
    return sorted_b_files

In [16]:
# this function comes from Chris Gordon's bgcArgoDMQC Python package (without changes)
def string_to_array(s, dim, encode='utf-8'):
    '''
    Args:
        s: input string or comment
        dim: netCDF dimension which the string will be made to fill using trailing whitespace
        encode (optional): encoding each character, default utf-8 (def. python netCDF format)

    Returns:
        numpy.array of single letters

    Author:
        Christopher Gordon
        Fisheries and Oceans Canada
        chris.gordon@dfo-mpo.gc.ca

    Change log:
        - 2021-07-05: initial commit
    '''

    M = dim.size - len(s)
    full_string = s + M*' '
    str_array = np.array([f'{c}'.encode(encode) for c in full_string])

    return str_array

In [17]:
# this function comes from Chris Gordon's bgcArgoDMQC Python package
# changes: added the iprof argument (original version runs a loop over all profiles)
#          added the incr_hist_dim argument (original version always increases the N_HISTORY dimension)
def update_history(nc, dct, iprof, incr_hist_dim=True):
    '''
    Update HISTORY_<PARAM> values in an Argo netCDF file
    '''
    hix = nc.dimensions['N_HISTORY'].size
    if not incr_hist_dim:
        hix -= 1
    for name, value in dct.items():
        nc[name][hix,iprof,:] = string_to_array(value, nc.dimensions[nc[name].dimensions[-1]])

In [18]:
def write_parameter_data_mode(bgc_file, this_iprof):
    '''Modify the PARAMETER_DATA_MODE variable in the BD file with the given handle.
    Set it to D for the current variable.'''
    ParameterList = bgc_file.variables["STATION_PARAMETERS"][this_iprof].data.astype(str)
    PDMarray = bgc_file.variables["PARAMETER_DATA_MODE"][this_iprof,:]
    for j in range(bgc_file.dimensions["N_PARAM"].size): 
        PARAMstr = ''.join(ParameterList[j]);
        if PARAMstr[:len(variable)] == variable: 
            PDMarray[j] = parameter_data_mode
        
    bgc_file.variables["PARAMETER_DATA_MODE"][this_iprof,:] = PDMarray
    # adjust the overall DATA_MODE as well (it may be 'D' already)
    bgc_file.variables["DATA_MODE"][this_iprof] = 'D'

In [19]:
def write_history(bgc_file, this_var, this_iprof, incr_hist_dim=True):
    '''Write global attributes and HISTORY* variables to the BD file with the given handle.
    Also writes out DATE_CREATION and DATE_UPDATE variables.'''
    # history_institution and history_reference must be set as global variables
    
    # Setting Global Attributes for BD file 
    # Argo Users Manual V3.41.1 §2.2.1

    # format: 'YYYY-MM-DDTHH:MM:SSZ creation' EG: '2023-03-13T19:25:43Z creation'
    bgc_file.history = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ creation")
    bgc_file.setncattr('comment_dmqc_operator', comment_dmqc_operator) # FIXME replace existing?
        
    history_step = "ARSQ" 
        # specific to Dmode. See ARGO User Manual section 3.12 Reference table 12: history steps codes 
    history_software= "SAGE" 
        # specific to Dmode. This code is institution dependent.
    history_software_release = "2023"
        # specific to Dmode. This code is institution dependent. 
        # Probably not even necessary to use for SAGE. I ended up using the year of the most recent update for SAGE.
        # SAGE does not appeare to have release numbers, saving update year as HISTORY_SOFTWARE_RELEASE 

    # Reference data that is being fed into SAGE. This code is institution dependent
    # See ARGO User Manual section 3.7 Reference table 7: history action codes
    history_action = "IP"
            
    history_parameter = this_var

    # Setting History Variables  
    # Using code from Christopher Gordon's library, but with added arguments:
    update_history(bgc_file, {"HISTORY_INSTITUTION": history_institution, # PM
                            "HISTORY_STEP": history_step, # specific to Dmode
                            "HISTORY_SOFTWARE": history_software, # specific to Dmode
                            "HISTORY_SOFTWARE_RELEASE": history_software_release, # specific to Dmode
                            "HISTORY_REFERENCE": history_reference[this_var], # Reference data that is being fed into SAGE, Institution dependent
                            "HISTORY_DATE": datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S"),
                            "HISTORY_ACTION": history_action,
                            "HISTORY_PARAMETER": history_parameter}, this_iprof, incr_hist_dim) # specific to Dmode
    # Setting DATE_CREATION (if it doesn't exist yet) and DATE_UPDATE variables; format: YYYYMMDDHHMISS (UTC)
    UTCcurrent = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
    if 'DATE_CREATION' in bgc_file.variables:
        date_creation = bgc_file.variables["DATE_CREATION"][:].tobytes().decode()
        if len(date_creation) != 14:            
            print(f'date_creation: {date_creation}')
            pdb.set_trace() # FIXME
            bgc_file.variables["DATE_CREATION"][:] = string_to_array(UTCcurrent, bgc_file.dimensions["DATE_TIME"])
    else:
        bgc_file.variables["DATE_CREATION"][:] = string_to_array(UTCcurrent, bgc_file.dimensions["DATE_TIME"])
    bgc_file.variables["DATE_UPDATE"][:] = string_to_array(UTCcurrent, bgc_file.dimensions["DATE_TIME"])

In [20]:
def get_variable(filename, var_name, this_iprof=0):
    '''Retrieve and return the JULD value from the given file.".'''
    b_file = netCDF4.Dataset(filename, 'r')
    value = b_file.variables[var_name][this_iprof].item()
    b_file.close()
    return value

In [21]:
def get_dim_len(filename, dim_name):
    '''Determine and return the length of the dimension named "dim_name".'''
    this_file = netCDF4.Dataset(filename, 'r')
    dim_len = len(this_file.dimensions[dim_name])
    this_file.close()
    return dim_len

In [22]:
def get_current_gain_offset_drift(adjustments, cycle):
    '''Find the row in dataframe adjustments with a station value that does 
    not exceed the given cycle. Return that row of the dataframe.'''
    # Filter the adjustments dataframe to find stations not larger than cycle
    filtered_adj = adjustments[adjustments['Station'] <= cycle]
    # Get the row with the highest number in the filtered dataframe
    if not filtered_adj.empty:
        # contains columns: Station, Gain, Offset, Drift
        return filtered_adj.iloc[-1]
    else: # this should never happen
        raise ValueError(f'cycle is too low: {cycle}')

In [23]:
def set_sci_calib_coefficients(current_adj, juld, juld_pivot):
    '''Fills the given values into the scientific calibration coefficient,
    which must be a string at most 256 characters long.'''
    # Sample output:
    # OFFSET = -2.9708; DRIFT = -0.9826; GAIN = 0.9300; JULD = 27120.3596; JULD_PIVOT = 26493.6594
    if current_adj is not None:
        if variable == 'PH_IN_SITU_TOTAL':
            str = 'PUMP_OFFSET = 0; '
        else:
            str = ''
        str += f'OFFSET = {current_adj["Offset"]:.4f}; DRIFT = {current_adj["Drift"]:.4f}; '
        str += f'GAIN = {current_adj["Gain"]:.4f}; JULD = {juld:.4f}; JULD_PIVOT = {juld_pivot:.4f}'
    else:
        if variable == 'PH_IN_SITU_TOTAL':
            str = 'PUMP_OFFSET = NAN; '
        else:
            str = ''
        str += f'OFFSET = NAN; DRIFT = NAN; '
        str += f'GAIN = NAN; JULD = NAN; JULD_PIVOT = NAN'
    return str

In [24]:
def write_scientific_calib(bgc_file, scientific_calibration_coefficient, this_sci_calib_eqn, this_iprof):
    '''Write the SCIENTIFIC_CALIB_* (DATE, COMMENT, EQUATION, and COEFFICIENT)
    variables in the BD file with the given handle.
    scientific_calibration_comment and scientific_calibration_equation must
    be assigned globally.'''
    # SCIENTIFIC_CALIB_DATE
    UTCcurrent = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
    #MS bgc_file.variables["SCIENTIFIC_CALIB_DATE"][iprof,:] = CGnetcdf.string_to_array(UTCcurrent, bgc_file.dimensions["DATE_TIME"])

    scientific_calibration_comment = sci_calib_comment[variable]
    scientific_calibration_equation = this_sci_calib_eqn

    # SCIENTIFIC_CALIB_COMMENT
    SciCalComArray = np.ma.empty(shape = (bgc_file.dimensions["STRING256"].size), dtype='|S1')
    SciCalComArray[:] = ''
    SciCalComArray.mask = True
    SciCalComArray[:len(scientific_calibration_comment)] = list(scientific_calibration_comment)

    # SCIENTIFIC_CALIB_EQUATION
    SciCalEquArray =  np.ma.empty(shape = (bgc_file.dimensions["STRING256"].size), dtype='|S1')
    SciCalEquArray[:] = ''
    SciCalEquArray.mask = True
    SciCalEquArray[:len(scientific_calibration_equation)] = list(scientific_calibration_equation)

    # SCIENTIFIC_CALIB_COEFFICIENT
    SciCalCoeArray = np.ma.empty(shape = (bgc_file.dimensions["STRING256"].size), dtype='|S1')
    SciCalCoeArray[:] = ''
    SciCalCoeArray.mask = True
    SciCalCoeArray[:len(scientific_calibration_coefficient)] = list(scientific_calibration_coefficient)

    ParameterList= bgc_file.variables["STATION_PARAMETERS"][this_iprof].data.astype(str)
    for j in range(bgc_file.dimensions["N_PARAM"].size): 
        PARAMstr= ''.join(ParameterList[j]);
        if PARAMstr[:len(variable)] == variable: 
            bgc_file.variables['SCIENTIFIC_CALIB_COMMENT'][this_iprof,0,j,] = SciCalComArray
            bgc_file.variables['SCIENTIFIC_CALIB_EQUATION'][this_iprof, 0, j,] = SciCalEquArray
            bgc_file.variables['SCIENTIFIC_CALIB_COEFFICIENT'][this_iprof, 0, j,] = SciCalCoeArray
            bgc_file.variables["SCIENTIFIC_CALIB_DATE"][this_iprof,0,j,:] = \
                string_to_array(UTCcurrent, bgc_file.dimensions["DATE_TIME"])

In [25]:
def interp_adj_err(adj_err, pres, iprof_src, iprof_dest):
    '''Interpolate the adjusted error from one pressure axis
    (index: iprof_src) to another (index: iprof_dest).
    Return the interpolated adjusted error.'''
    # determine finite values
    pres_src = pres[iprof_src,:]
    adj_err_src = adj_err[iprof_src,:]
    pres_good = pres_src[~pres_src.mask & ~adj_err_src.mask]
    adj_err_good = adj_err_src[~pres_src.mask & ~adj_err_src.mask]
    # https://docs.scipy.org/doc/scipy/tutorial/interpolate/extrapolation_examples.html
    natural = CubicSpline(pres_good, adj_err_good, bc_type='natural')
    adj_err_intp = natural(pres[iprof_dest,:])
    pres_dest = pres[iprof_dest,:]
    adj_err[iprof_dest] = np.ma.masked_array(adj_err_intp, mask=pres_dest.mask, fill_value=adj_err.fill_value)
    return adj_err

In [26]:
def get_total_offset(current_adj, this_juld, juld_pivot):
    '''Compute and return the total offset for the given Julian day.'''
    # total_offset = OFFSET + DRIFT(JULD-JULD_PIVOT)/365]
    if current_adj is not None:
        return current_adj['Offset'] + current_adj['Drift'] * (this_juld - juld_pivot) / 365.0;
    else:
        return np.nan

In [38]:
def write_doxy_adjusted(bgc_file, gain):
    '''Read DOXY values, compute DOXY_ADJUSTED = DOXY * gain, 
    and write them to the BD file with the given file handle. 
    Also write DOXY_ADJUSTED_QC values.
    Return the masked array DOXY_Adjusted_Array.'''    
    print(type(this_iprof)) 
    print(this_iprof)
    doxy = bgc_file.variables['DOXY'][this_iprof,:]
    doxy_adjusted = doxy * gain;
    doxy_adjusted_qc = np.empty(shape = (bgc_file.dimensions["N_LEVELS"].size), dtype='|S1')
    doxy_adjusted_qc[:] = ' ' # missing value
    doxy_adjusted_qc[~doxy_adjusted.mask] = '1'
    
    bgc_file.variables["DOXY_ADJUSTED"][this_iprof,:] = doxy_adjusted
    bgc_file.variables["DOXY_ADJUSTED_QC"][this_iprof,:] = doxy_adjusted_qc
    return doxy_adjusted

In [39]:
def write_doxy_adjusted_error(bgc_file, doxy_adj_err_mbar, psal, temp, pres, dens, doxy_adj):
    '''Given the (scalar) value of doxy_adj_err_mbar (in mbar), compute it for all
    depth levels in umol/kg and write it to the given bgc_file.
    Return value is in umol/l.'''
    idx_good = np.where(~np.ma.getmask(psal) & ~np.ma.getmask(doxy_adj))
    # initialize the error with its full size
    doxy_adj_error = np.ma.masked_array(psal.shape[0]*[0.0], mask=np.ma.getmask(psal) | np.ma.getmask(doxy_adj), fill_value=99999.0)
    error = bgcArgoDMQC.calc_fixed_doxy_adjusted_error(psal[idx_good], temp[idx_good], pres[idx_good], 
                                                       fix_err=doxy_adj_err_mbar)
    doxy_adj_error[idx_good] = error
    # convert to umol/kg
    doxy_adj_error_umol_kg = doxy_adj_error * 1e3 / dens
    bgc_file.variables["DOXY_ADJUSTED_ERROR"][this_iprof,:] = doxy_adj_error_umol_kg

In [40]:
def write_nitrate_adjusted(bgc_file, offset, gain, this_iprof):
    '''Read NITRATE values, compute 
    NITRATE_ADJUSTED = [NITRATE - offset] / gain, 
    and write them to the BD file with the given file handle.
    Full equation:
    NITRATE_ADJUSTED=[NITRATE-[OFFSET + DRIFT(JULD-JULD_PIVOT)/365]]/GAIN
    offset = OFFSET + DRIFT(JULD-JULD_PIVOT)/365
    Also write NITRATE_ADJUSTED_QC values.
    Return the masked array NITRATE_Adjusted_Array.'''
    nitrate = bgc_file.variables['NITRATE'][this_iprof,:]
    nitrate_qc = bgc_file.variables['NITRATE_QC'][this_iprof,:]
    nitrate_adjusted = (nitrate - offset) / gain
    nitrate_adjusted_qc = np.empty(shape = (bgc_file.dimensions["N_LEVELS"].size), dtype='|S1')
    nitrate_adjusted_qc[:] = ' ' # use missing value as default
    nitrate_adjusted_qc[~nitrate_adjusted.mask] = '1'
    nitrate_adjusted_qc[~nitrate_qc.mask & nitrate_adjusted.mask] = '9'
    # output to netcdf file
    bgc_file.variables["NITRATE_ADJUSTED"][this_iprof,:] = nitrate_adjusted
    bgc_file.variables["NITRATE_ADJUSTED_QC"][this_iprof,:] = nitrate_adjusted_qc
    return nitrate_adjusted

In [41]:
def write_nitrate_adjusted_error(bgc_file, nitr_adj, pres, doxy_adj_err, this_iprof):
    '''Given the values of doxy_adj_err (in umol/kg), compute NITRATE_ADJUSTED_ERROR 
    for all depth levels in umol/kg and write it to the given bgc_file.
    NITRATE_ADJUSTED_ERROR = 1 µmol/kg + DOXY_ADJUSTED_ERROR/10
    '''
    # initialize the error with its full size, masked everywhere
    nitr_adj_err = np.ma.masked_array(np.zeros(pres.shape), mask=True, fill_value=99999.0)
    idx_good = np.where(~doxy_adj_err[this_iprof,:].mask  & ~nitr_adj[:].mask)
    nitr_adj_err[this_iprof,idx_good] = 1.0 + 0.1 * doxy_adj_err[this_iprof,idx_good]
    bgc_file.variables["NITRATE_ADJUSTED_ERROR"][:,:] = nitr_adj_err

In [42]:
def compute_tcor(pres, temp, this_iprof, ref_depth=reference_depth):
    '''
    TCOR = (TREF+273.15) / (TEMP+273.15)
    REF = TEMP at 1500m
    '''
    # determine the pressure level (for this_iprof) that's closest to the reference depth
    idx_ref = np.argmin(abs(pres[this_iprof,:] - ref_depth))
    # determine temperature at this pressure level
    tref = temp[this_iprof,idx_ref]
    tk = 273.15
    tcor = (tref + tk) / (temp[this_iprof,:] + tk)
    # determine reference pressure
    if abs(pres[this_iprof,idx_ref] - ref_depth) < 50:
        pres_ref = ref_depth
    else:
        # round to the nearest multiple of 50 dbar
        pres_ref = 50 * round(pres[this_iprof,idx_ref] / 50)
    return tcor, pres_ref # tcor is a vector

In [43]:
def write_ph_adjusted(bgc_file, offset, tcor, gain, this_iprof):
    '''Read PH_IN_SITU_TOTAL values, compute 
    PH_IN_SITU_TOTAL_ADJUSTED = [PH_IN_SITU_TOTAL - offset * tcor] / gain,
    and write them to the BD file with the given file handle.
    FIXME: non-zero values of PUMP_OFFSET are not yet handled
    Full equation:
    PH_IN_SITU_TOTAL_ADJUSTED=[PH_IN_SITU_TOTAL+[PUMP_OFFSET - [OFFSET + DRIFT(JULD-JULD_PIVOT)/365]*TCOR]]/GAIN
    offset = OFFSET + DRIFT*(JULD-JULD_PIVOT)/365
    TCOR=(TREF+273.15)./(TEMP+273.15)
    TREF = TEMP at 1500m (in general: temperature at reference pressure)
    Also write PH_IN_SITU_TOTAL_ADJUSTED_QC values.
    Return the masked array PH_IN_SITU_TOTAL_Adjusted_Array.'''
    ph = bgc_file.variables['PH_IN_SITU_TOTAL'][this_iprof,:]
    ph_adjusted = (ph - offset * tcor) / gain
    ph_adjusted_qc = np.empty(shape = (bgc_file.dimensions["N_LEVELS"].size), dtype='|S1')
    ph_adjusted_qc[:] = ' ' # use missing value as default
    ph_adjusted_qc[~ph_adjusted.mask] = '2' # FIXME
    # output to netcdf file
    bgc_file.variables["PH_IN_SITU_TOTAL_ADJUSTED"][this_iprof,:] = ph_adjusted
    bgc_file.variables["PH_IN_SITU_TOTAL_ADJUSTED_QC"][this_iprof,:] = ph_adjusted_qc
    return ph_adjusted

In [44]:
def write_ph_adjusted_error(bgc_file, ph_adj, pres, doxy_adj_err, this_iprof):
    '''Given the values of doxy_adj_err (in umol/kg), compute PH_IN_SITU_TOTAL_ADJUSTED_ERROR 
    for all depth levels in umol/kg and write it to the given bgc_file.
    PH_IN_SITU_TOTAL_ADJUSTED_ERROR = 0.01 + 0.0016*DOXY_ADJUSTED_ERROR
    '''
    # initialize the error with its full size, masked everywhere
    ph_adj_err = np.ma.masked_array(np.zeros(pres.shape), mask=True, fill_value=99999.0)
    idx_good = np.where(~doxy_adj_err[this_iprof,:].mask & ~ph_adj[:].mask)
    idx_no_doxy_error = np.where(doxy_adj_err[this_iprof,:].mask & ~ph_adj[:].mask)
    # if this_iprof != iprof_doxy, function interp_adj_err must have filled in doxy_adj_err with the correct profile for pH
    ph_adj_err[this_iprof,idx_good] = 0.01 + 0.0016 * doxy_adj_err[this_iprof,idx_good]
    ph_adj_err[this_iprof,idx_no_doxy_error] = 0.01 + 0.0016 * 5
        
    bgc_file.variables["PH_IN_SITU_TOTAL_ADJUSTED_ERROR"][:,:] = ph_adj_err

In [45]:
def write_bad_profile(bgc_file, this_iprof):
    '''This function is only used if all values in a profile are bad, 
    probably due to a failure of the sensor.
    Includes setting of _QC values to 9 for base and adjusted variable'''
    base_var = bgc_file.variables[variable][this_iprof,:] # only needed for size
    adjusted_var = ma.masked_all_like(base_var)
    # QC flag for the raw variable
    base_qc = np.empty(shape = (bgc_file.dimensions["N_LEVELS"].size), dtype='|S1')
    base_qc[:] = ' ' # use missing value as default
    base_qc[~base_var.mask] = '4' # all existing values are bad
    bgc_file.variables[f"{variable}_QC"][this_iprof,:] = base_qc
    # use the same QC values for the adjusted values
    bgc_file.variables[f"{variable}_ADJUSTED_QC"][this_iprof,:] = base_qc
    
    # fill the masked array with NaN values
    adjusted_var.fill_value = np.nan
    
    # output to netcdf file
    bgc_file.variables[f"{variable}_ADJUSTED"][this_iprof,:] = adjusted_var

In [46]:
def get_profile(filename):
    '''Extract the profile index from filename, return it as an int.'''
    profile = filename[-6:-3]
    return int(profile)

In [47]:
# extract cycle, gain, offset, and drift values from ODV*QC.TXT file
adjustments = {}
for var in variables: 
    adjustments[var] = get_gain_offset_drift_odvqc(odv_filename, variable=var) # return value is a DataFrame

In [48]:
# loop over all BR and BD files - they must be present in "float_dir" already
all_bd_files = glob.glob(f'{float_dir}/BD*.nc')
all_bd_files.sort()
all_br_files = glob.glob(f'{float_dir}/BR*.nc')
all_br_files.sort()

print(glob.glob(f'{float_dir}/BD*.nc'))

# if both BR and BD file exist for the same profile, only the BD file will be used
sorted_b_files = organize_b_files(all_bd_files, all_br_files)
print(f'{len(sorted_b_files)} relevant B files found in {float_dir}')
#print(f'Sorted files: {sorted_b_files}')
juld_init = get_variable(sorted_b_files[0], 'JULD')
print(f'JULD_INIT: {juld_init}')
    
new_bd_files = list()                                 
# need to keep track of previous JULD values to determine JULD_PIVOT from cycle
juld = {} 

# how many profiles do the B files have?
n_prof = get_dim_len(sorted_b_files[0], 'N_PROF')

changed_iprof = [False] * n_prof

# outer loop over variables
for variable in variables:
    # custom settings - bad profiles determined during DMQC process
    if floatid == 4903499 and variable == 'PH_IN_SITU_TOTAL':
        idx_prof_failure = 20 # first profile for which pH is completely bad
    elif floatid == 4903500 and variable == 'PH_IN_SITU_TOTAL':
        idx_prof_failure = 27 # first profile for which pH is completely bad
    elif floatid == 4903622 and variable == 'PH_IN_SITU_TOTAL':
        idx_prof_failure = 5 # first profile for which pH is completely bad
    elif floatid == 4903624 and variable == 'PH_IN_SITU_TOTAL':
        idx_prof_failure = 65 # first profile for which pH is completely bad 
    elif floatid == 4903622 and variable == 'PH_IN_SITU_TOTAL':
        idx_prof_failure = 5 # first profile for which pH is completely bad
    elif floatid == 4903624 and variable == 'NITRATE':
        idx_prof_failure = 67 # first profile for which Nitrate is completely bad
    else:
        idx_prof_failure = 100000 # reset to "never"
    print(adjustments[variable])
    this_iprof = iprof[variable]
    print(f'Taking variable {variable} from profile {this_iprof+1}') # count from 1 for humans
    # always increase N_HISTORY for the first variable; for other variables, increase N_HISTORY
    # only if an entry for the current profile was made for another variable already
    if not any(changed_iprof) or changed_iprof[this_iprof]:
        incr_hist_dim = True
    else:
        incr_hist_dim = False
    changed_iprof[this_iprof] = True
    print(f'N_HISTORY will be increased: {incr_hist_dim}')
    
    # inner loop over files
    for bgc_filename in sorted_b_files:
        print(f'Processing {bgc_filename}')
        idx_profile = int(bgc_filename[-6:-3])
        cycle = get_variable(bgc_filename, 'CYCLE_NUMBER')
        if floatid == 4903622 and variable == 'PH_IN_SITU_TOTAL' and cycle >= 8:
            print(f'Float {floatid} pH diabled for cycles {cycle}+ .\nEnd pH processing')  
            break #pH sensor was disabled, does not exist for cycle 8 and after
        if floatid == 4903624 and variable == 'PH_IN_SITU_TOTAL' and cycle >= 68:
            print(f'Float {floatid} pH diabled for cycles {cycle}+ .\nEnd pH processing')  
            break #pH sensor was disabled, does not exist for cycle or 68 and after
    
            
        juld[cycle] = get_variable(bgc_filename, 'JULD')
        compute_adjusted = True
        
        # check if sensor has gone bad
        if idx_profile >= idx_prof_failure:
            print(f'Profile {idx_profile} for {variable} is bad!')
            profile_var_qc = 'F'
            compute_adjusted = False
           # if variable != 'PH_IN_SITU_TOTAL':
           #     raise ValueError('case not yet handled')
            juld_pivot = np.nan 
            current_adj = None # flag for function set_sci_calib_coefficients
        else:
            profile_var_qc = 'A' # FIXME
            current_adj = get_current_gain_offset_drift(adjustments[variable], cycle)
            juld_pivot = juld[int(current_adj['Station'])]

        w_bgc_filename = create_working_bd_file(bgc_filename)    
        bgc_file = netCDF4.Dataset(w_bgc_filename, 'a')
        write_history(bgc_file, variable, this_iprof, incr_hist_dim)
        write_parameter_data_mode(bgc_file, this_iprof)
        sci_calib_coeffs = set_sci_calib_coefficients(current_adj, juld[cycle], juld_pivot)
        #DEBUG print(sci_calib_coeffs)

        bgc_file.variables["DATA_STATE_INDICATOR"][this_iprof] = np.ma.array(data_state_indicator, 
                                                                    mask=[False, False, True, True], 
                                                                    dtype='|S1')

        # need to determine which pTS profile index to use FIXME this comment
        phys_filename = get_phys_filename(bgc_filename)
        pres_phys_raw = get_phys_raw_pres(phys_filename)
        pres_bgc = bgc_file.variables['PRES'][:,:]
        # both profiles should have matching pressure values in phys and bgc files
        if max(abs(pres_phys_raw[0,:]-pres_bgc[0,:])) > 0.0 or max(abs(pres_phys_raw[1,:]-pres_bgc[1,:])) > 0.0:
            raise ValueError('mismatch in pressure values')
        #FIXME UNNEEDED iprof_phys = get_iprof_phys(pres_phys_raw, pres_bgc)

        if variable == 'DOXY' or variable == 'PH_IN_SITU_TOTAL':
            # extract variables from corresponding physical profile file
            dens, psal, temp = get_dens(phys_filename)

        if variable != 'DOXY':
            # adjusted error of nitrate and pH is derived from DOXY_ADJUSTED_ERROR
            doxy_adj_err = bgc_file.variables['DOXY_ADJUSTED_ERROR'][:,:]
            if this_iprof != iprof['DOXY']:
                # interpolate DOXY_ADJUSTED_ERROR to the other pressure axis
                doxy_adj_err = interp_adj_err(doxy_adj_err, pres_bgc, iprof['DOXY'], this_iprof)

        # compute total_offset for this profile
        total_offset = get_total_offset(current_adj, juld[cycle], juld_pivot)
        #DEBUG print(total_offset)
        this_sci_calib_eqn = sci_calib_eqn[variable]
        
        if compute_adjusted:
            if variable == 'DOXY':
                # compute DOXY_ADJUSTED from DOXY and gain value and write it to BD file 
                gain = 1.0489
                doxy_adjusted = write_doxy_adjusted(bgc_file, gain)
                # calculate DOXY_ADJUSTED_ERROR in umol/kg    
                write_doxy_adjusted_error(bgc_file, doxy_adj_err, psal[iprof_phys,:], temp[iprof_phys,:], 
                    pres_bgc[iprof,:], dens_phys[iprof_phys,:], doxy_adjusted)
            elif variable == 'NITRATE':
                # compute and write NITRATE_ADJUSTED    
                nitr_adjusted = write_nitrate_adjusted(bgc_file, total_offset, current_adj['Gain'], this_iprof)
                # calculate NITRATE_ADJUSTED_ERROR in umol/kg from DOXY_ADJUSTED_ERROR and write it to BD file   
                write_nitrate_adjusted_error(bgc_file, nitr_adjusted, pres_bgc, doxy_adj_err, this_iprof)
            elif variable == 'PH_IN_SITU_TOTAL':
                tcor, pres_ref = compute_tcor(pres_phys_raw, temp, this_iprof)
                ph_adjusted = write_ph_adjusted(bgc_file, total_offset, tcor, current_adj['Gain'], this_iprof)
                if abs(pres_ref - reference_depth) > 0.1:
                    regex = re.compile(rf'({reference_depth})\s*m')
                    match_obj = regex.search(this_sci_calib_eqn)
                    if match_obj:
                        this_sci_calib_eqn = this_sci_calib_eqn.replace(match_obj.group(1), f'{pres_ref}')
                    else:
                        raise ValueError('no string with reference depth found in sci calib equation')
                # calculate PH_IN_SITU_TOTAL_ADJUSTED_ERROR in umol/kg from DOXY_ADJUSTED_ERROR and write it to BD file   
                write_ph_adjusted_error(bgc_file, ph_adjusted, pres_bgc, doxy_adj_err, this_iprof)
            else:
                raise ValueError('not yet coded')
        else:
            write_bad_profile(bgc_file, this_iprof) # writes _ADJUSTED, _ADJUSTED_QC, _QC values
        # reference depth that was used for pH may differ from default; content may have been changed
        #DEBUG print(sci_calib_coeffs)
        write_scientific_calib(bgc_file, sci_calib_coeffs, this_sci_calib_eqn, this_iprof)

        # assign a new overall profile QC flag
        prof_qc_name = f'PROFILE_{variable}_QC'
        bgc_file.variables[prof_qc_name][this_iprof] = profile_var_qc

        # fix the vertical sampling scheme entry FIXME
        if variable == variables[0]:
            str_dim = bgc_file.dimensions[bgc_file['VERTICAL_SAMPLING_SCHEME'].dimensions[-1]]
            vss0 = 'Primary sampling: averaged [binned data sampled at 1.0 Hz from a SBE41CP; nominal 2 dbar to 1000 bar, 4 dbar below 1000 dbar]'
            bgc_file['VERTICAL_SAMPLING_SCHEME'][0,:] = string_to_array(vss0, str_dim)
            vss1 = 'Secondary sampling: discrete [from the same SBE41CP]'
            bgc_file['VERTICAL_SAMPLING_SCHEME'][1,:] = string_to_array(vss1, str_dim)            
            vss = bgc_file['VERTICAL_SAMPLING_SCHEME'][:]

        bgc_file.close()
        new_bd_files.append(w_bgc_filename)

        print(f'Renaming {w_bgc_filename} to {bgc_filename}')
        os.rename(w_bgc_filename, bgc_filename)

[]
129 relevant B files found in /Users/madison.soden/Documents/MATLAB/ARGO_PROCESSING/DATA/ARGO_REPO/4903622/QC/
JULD_INIT: 26210.909594934306
   Station    Gain  Offset   Drift
0      1.0  1.0521     0.0 -0.0009
1    130.0  1.0489     0.0  0.0000
Taking variable DOXY from profile 2
N_HISTORY will be increased: True
Processing /Users/madison.soden/Documents/MATLAB/ARGO_PROCESSING/DATA/ARGO_REPO/4903622/QC/BR4903622_001.nc
<class 'int'>
1


NameError: name 'doxy_adj_err' is not defined

In [49]:
print('These BD files were created or modified:')
print(new_bd_files)

These BD files were created or modified:
[]


In [ ]:
all_bd_files = glob.glob(f'{float_dir}BD*{floatid}_*.nc')
print(all_bd_files)

In [ ]:
#bgc_file = netCDF4.Dataset('/Users/madison.soden/Desktop/4903622_V1_Nit_pH/BD4903622_095.nc', 'a')
#bgc_file.variables["PROFILE_NITRATE_QC"][1]= bytes(' ', 'utf-8')
#bgc_file.variables["PROFILE_NITRATE_QC"]
#bgc_file.close()

In [ ]:
#bgc_file.variables["PROFILE_RPHASE_DOXY_QC"]

In [ ]:
#bgc_file = netCDF4.Dataset('/Users/madison.soden/Desktop/4903625_sus/BD4903625_017.nc', 'a')
#bgc_file.variables["PROFILE_PH_IN_SITU_TOTAL_QC"][1]= bytes(' ', 'utf-8')
#bgc_file.variables["PROFILE_PH_IN_SITU_TOTAL_QC"]
#bgc_file.close()

In [ ]:
glob.glob(f'{float_dir}/BR*.nc')